In [20]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
client = openai.Client()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
assistant = client.beta.assistants.create(
    name="Tutor de tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

In [23]:
pergunta = """
Em um sistema de cache com mapeamento direito, dado uma memória principal de uma cache 
de 256 KB e blocos de 64 bytes
- Quantas linhas existem na cache?
- onde o bloco de endereço 0x1A2B3C será mapeado na cache?
"""

In [24]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [25]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [26]:
run.status

'queued'

In [27]:
# Aguarda a thread ser executada
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [28]:
# Verificar resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro: {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_oacqIebjglSd6oDWMdPezUil', assistant_id='asst_Lo5QwDya3RqOXGAkTmrnaSjs', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A2B3C será mapeado na linha 2876 da cache.'), type='text')], created_at=1746577826, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_hLTYqgUYHK9SDdWuhCWO6KPQ', status=None, thread_id='thread_RzAPsxT5vYrNlffzjMtdmUJw'), Message(id='msg_WPJBCRdLuDdAFKhaQPG6OQgh', assistant_id='asst_Lo5QwDya3RqOXGAkTmrnaSjs', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Os bits de indexação do endereço 0x1A2B3C são: 101100111100\n\nIsso significa que esses bits são usados para determinar a linha na cache onde o bloco será mapeado. Vamos converter esses bits para decimal para identificar a linha exata na cache.'), type='text')], created_at=1746577824, incomplete_a

In [29]:
print(mensagens.data[0].content[0].text.value)

O bloco de endereço 0x1A2B3C será mapeado na linha 2876 da cache.


In [31]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [32]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_cals":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


====Step message_creation
Para calcular a quantidade de linhas na cache com mapeamento direto, precisamos dividir o tamanho da cache pelo tamanho de cada bloco.

Dado que:
- Tamanho da cache = 256 KB = 256 * 1024 bytes
- Tamanho de bloco = 64 bytes

Podemos calcular o número total de linhas na cache. Vamos fazer isso primeiro. Em seguida, calcularemos a posição em que o bloco de endereço 0x1A2B3C será mapeado na cache.

====Step tool_calls

====Step message_creation
A cache com mapeamento direto possui 4096 linhas.

Agora, para determinar onde o bloco de endereço 0x1A2B3C será mapeado na cache, precisamos analisar o endereço do bloco em binário e então identificar os bits relevantes para o mapeamento.

O endereço 0x1A2B3C em hexadecimal é:

- 0x1A2B3C = 000110100010101100111100

Vamos converter este endereço hexadecimal para binário e identificar os bits relevantes para o mapeamento.

====Step tool_calls

====Step message_creation
O endereço do bloco 0x1A2B3C em binário é: 00011010001